In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

# Installing pillow v4.0.
from PIL import Image
!pip install Pillow==4.0.0
!pip install image

tcmalloc: large alloc 1073750016 bytes == 0x5c6ac000 @  0x7f986fdb61c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True
    100% |████████████████████████████████| 5.6MB 3.9MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
    100% |████████████████████████████████| 7.3MB 4.8MB/s 


In [3]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

import matplotlib.pyplot as plt
%matplotlib inline

###########################################################################################
# Adapted from: https://github.com/L1aoXingyu/pytorch-beginner/tree/master/08-AutoEncoder #
###########################################################################################

if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

# Parameters.
num_epochs = 100
batch_size = 64
learning_rate = 1e-3

# Defining normalization routine to images.
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Instancing Dataloader and Dataset.
dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Defining model.
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(8, 4, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4, 8, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        
        z = self.encoder(x)
        out = self.decoder(z)
        return out


# Instancing Model
model = autoencoder().cuda()

# Defining Criterion
criterion = nn.MSELoss()

# Defining Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# Iterating over epochs.
for epoch in range(num_epochs):

    # Iterating over batches.
    for it, data in enumerate(dataloader):

        # Reading data and casting to GPU.
        img, _ = data
        img = Variable(img).cuda()
        
        # Forward.
        output = model(img)
        loss = criterion(output, img)
        
        # Backward.
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        # Plotting first batch of each epoch.
        if it == 0:
            
            fig, ax = plt.subplots(8, 8, figsize=(12, 12))
            for i in range(8):
                for j in range(8):
                    ax[i, j].imshow(output.cpu()[i + 4 * j].detach().numpy()[0])
                    ax[i, j].set_yticks([])
                    ax[i, j].set_xticks([])
            plt.show()
            
    # Printing log.
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data[0]))

# Saving model to disk.
torch.save(model.state_dict(), './conv_autoencoder.pth')